In [1]:
import sqlite3

In [7]:
import requests 

In [8]:
import pandas as pd

In [2]:
con = sqlite3.connect("chinook.db")

In [3]:
cur = con.cursor()

In [9]:
res = cur.execute("select InvoiceId, CustomerId, InvoiceDate from invoices;")


In [10]:
output = res.fetchall()

In [15]:
invoices = pd.DataFrame(output, columns=["InvoiceId", "CustomerId", "InvoiceDate"])

In [16]:
invoices["InvoiceDate"].dtype
# o means object

dtype('O')

In [17]:
invoice_date = pd.to_datetime(invoices["InvoiceDate"])

In [18]:
invoice_date.dtype
#datatype is now a date time object

dtype('<M8[ns]')

In [28]:
invoice_date.dt.strftime("%m/%d/%Y")

0      01/01/2009
1      01/02/2009
2      01/03/2009
3      01/06/2009
4      01/11/2009
          ...    
407    12/05/2013
408    12/06/2013
409    12/09/2013
410    12/14/2013
411    12/22/2013
Name: InvoiceDate, Length: 412, dtype: object

In [31]:
invoices["InvoiceYear"]=invoice_date.dt.strftime("%Y")

In [33]:
invoices.head(3)

,InvoiceId,CustomerId,InvoiceDate,InvoiceYear
0,1,2,2009-01-01 00:00:00,2009
1,2,4,2009-01-02 00:00:00,2009
2,3,8,2009-01-03 00:00:00,2009


### . groupby

to create groups and valeu by groups 
- in SQLite: select ... from ... group by

In [ ]:
#SELECT CustomerId, COUNT(InvoiceId) from invoices GROUP BY CustomerId

In [37]:
invoices.groupby(['CustomerId',"InvoiceYear"])["InvoiceId"].size()

CustomerId  InvoiceYear
1           2010           3
            2011           1
            2012           2
            2013           1
2           2009           3
                          ..
58          2013           3
59          2009           2
            2010           1
            2011           2
            2012           1
Name: InvoiceId, Length: 232, dtype: int64

In [39]:
con = sqlite3.connect("chinook.db")
cur = con.cursor()
res = cur.execute("select CustomerId, Company, Country, SupportRepId from customers;")
customers = res.fetchall()
customers = pd.DataFrame(customers, 
columns=["CustomerId", "Company", "Country", 
"SupportRepId"])

In [43]:
#merge (join) 'invoices' and 'customers'
# - "CustomerId" is the column should be matched

#sqlite3: select * from invoices join customers
# on invoices.CustomerId=custoemrs.CustomerId

merged= invoices.merge(customers, left_on="CustomerId", right_on="CustomerId")

In [44]:
merged


,InvoiceId,CustomerId,InvoiceDate,InvoiceYear,Company,Country,SupportRepId
0,1,2,2009-01-01 00:00:00,2009,None,Germany,5
1,2,4,2009-01-02 00:00:00,2009,None,Norway,4
2,3,8,2009-01-03 00:00:00,2009,None,Belgium,4
3,4,14,2009-01-06 00:00:00,2009,Telus,Canada,5
4,5,23,2009-01-11 00:00:00,2009,None,USA,4
...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,2013,None,USA,5
408,409,29,2013-12-06 00:00:00,2013,None,Canada,3
409,410,35,2013-12-09 00:00:00,2013,None,Portugal,4
410,411,44,2013-12-14 00:00:00,2013,None,Finland,3


In [40]:
customers

,CustomerId,Company,Country,SupportRepId
0,1,Embraer - Empresa Brasileira de Aeronáutica S.A.,Brazil,3
1,2,None,Germany,5
2,3,None,Canada,3
3,4,None,Norway,4
4,5,JetBrains s.r.o.,Czech Republic,4
5,6,None,Czech Republic,5
6,7,None,Austria,5
7,8,None,Belgium,4
8,9,None,Denmark,4
9,10,Woodstock Discos,Brazil,4


In [53]:
con = sqlite3.connect("chinook.db")
cur = con.cursor()
res = cur.execute("select EmployeeId, LastName, FirstName, Title from employees;")
output = res.fetchall()
emp = pd.DataFrame(output, columns=["EmployeeId", "LastName", "FirstName", "Title"])

In [55]:
#merge (join) 'invoices' and 'customers'
# - "CustomerId" is the column should be matched

#sqlite3: select * from invoices join customers
# on invoices.CustomerId=custoemrs.CustomerId

merged2= customers.merge(emp, left_on="SupportRepId", right_on="EmployeeId")

In [56]:
merged2

,CustomerId,Company,Country,SupportRepId,EmployeeId,LastName,FirstName,Title
0,1,Embraer - Empresa Brasileira de Aeronáutica S.A.,Brazil,3,3,Peacock,Jane,Sales Support Agent
1,2,None,Germany,5,5,Johnson,Steve,Sales Support Agent
2,3,None,Canada,3,3,Peacock,Jane,Sales Support Agent
3,4,None,Norway,4,4,Park,Margaret,Sales Support Agent
4,5,JetBrains s.r.o.,Czech Republic,4,4,Park,Margaret,Sales Support Agent
5,6,None,Czech Republic,5,5,Johnson,Steve,Sales Support Agent
6,7,None,Austria,5,5,Johnson,Steve,Sales Support Agent
7,8,None,Belgium,4,4,Park,Margaret,Sales Support Agent
8,9,None,Denmark,4,4,Park,Margaret,Sales Support Agent
9,10,Woodstock Discos,Brazil,4,4,Park,Margaret,Sales Support Agent
